# 6.804 Final Project

Meia Alsup, Sravya Bhamidipati, Anelise Newman

In [95]:
from string import digits
import json
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from random import randrange

from IPython.display import Image, display
from IPython.core.display import HTML 
import urllib.request as request

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import nltk


In [2]:
 #ocr = pkl.load(open('raw_ocr_output.pickle','rb'))
#less_ocr = {key: ocr[key] for i, key in enumerate(ocr) if i < 100}
#less_ocr

In [4]:
#f = open('icon_annotations_testing.pickle','rb')
#icons = pkl.load(f)
#f.close()

## Clean input to just English words



In [10]:
f = open('google_text_extraction_output.pckl','rb')
infographic_name_to_words = pkl.load(f)
f.close()

print('We have extracted words from %d infographics.'%len(infographic_name_to_words))

We have extracted words from 62361 infographics.


In [11]:
def contains_english(set_of_words):
    english = set(['the', 'she', 'different', 'time', 'and', 'client', 'sports', 'grammar', 'family', 'report'])
    spanish = set(['el', 'es', 'cuando', 'esta', 'malo', 'que', 'tiene', 'para'])
    if bool(set(set_of_words) & spanish):
        return False
    if bool(set(set_of_words) & english):
        return True
    return False

In [12]:
english  = {key: val for key, val in infographic_name_to_words.items() if contains_english(val)}
len(english)

46952

In [11]:
json.dump(english, open('filtered.json', 'w'))
json.dump([key for key, val in english.items()], open('filtered_keys.json', 'w'))

In [47]:

newEnglish = {}
newList = []
stemmer = PorterStemmer()
for url, wordList in english.items():
    thisList = []
    for word in wordList:
        if len(word) > 2 and word.isalpha() and word not in STOPWORDS and wordnet.synsets(word):
            thisList.append(word.lower())
            #thisList.append(stemmer.stem(WordNetLemmatizer().lemmatize(word.lower(), pos='v')))
    newList.append(thisList)
    newEnglish[url] = thisList

In [39]:
NUM_TOPICS = 20

In [40]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(newList)
common_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

common_corpus = [common_dictionary.doc2bow(text) for text in newList]
# Train the model on the corpus.
id2token = {val:key for key, val in common_dictionary.token2id.items()}

In [41]:
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=NUM_TOPICS)

In [42]:
for i in range(NUM_TOPICS):
    terms = lda.get_topic_terms(i, 20)
    print(f'Topic number {i}')
    val = {id2token[t[0]]: t[1] for t in terms}
    #print(f'Top 20 terms: {val}')
    print(f'Top 20 terms: {val.keys()}')

Topic number 0
Top 20 terms: dict_keys(['home', 'price', 'market', 'real', 'average', 'property', 'value', 'estate', 'money', 'sales', 'house', 'prices', 'buy', 'new', 'stock', 'buying', 'sell', 'london', 'homes', 'investment'])
Topic number 1
Top 20 terms: dict_keys(['social', 'media', 'marketing', 'content', 'customer', 'customers', 'people', 'email', 'use', 'twitter', 'brand', 'business', 'post', 'marketers', 'service', 'page', 'blog', 'share', 'create', 'like'])
Topic number 2
Top 20 terms: dict_keys(['million', 'united', 'states', 'world', 'new', 'years', 'number', 'south', 'population', 'state', 'countries', 'north', 'year', 'people', 'national', 'total', 'average', 'america', 'american', 'age'])
Topic number 3
Top 20 terms: dict_keys(['users', 'online', 'internet', 'google', 'mobile', 'search', 'web', 'million', 'video', 'website', 'site', 'billion', 'social', 'use', 'user', 'sites', 'digital', 'content', 'data', 'people'])
Topic number 4
Top 20 terms: dict_keys(['energy', 'wate

In [43]:
saved = lda.save('saved_lda_15_topics')

## Intersction with infographics that have labeled bounding boxes

In [ ]:
# annotations from task where participants marked bounding boxes around all icons on an infographic
icon_dict = pkl.load(open( "icon_annotations_all.pickle", "rb") )
icons_english  = set(icon_dict.keys())

In [17]:
base='http://thumbnails.visually.netdna-cdn.com'

In [13]:
intersection = set([url_ for url_ in english.keys() if url_ in icons_english])

In [15]:
len(intersection)

1079

In [18]:
for url_ in intersection:
    #display(Image(url=f'{base}/{url_}'))
    request.urlretrieve(f'{base}/{url_}', f'images/{url_}')

## See category labels for chosen inforgraphics


In [65]:
lda = LdaModel.load('saved_lda_20_topics')

In [82]:
categories = {
    0: 'healthcare',
    1: 'economy and government',
    2: 'work and positivity',
    3: 'celebrations and family',
    4: 'Internet and social media',
    5: 'international world',
    6: 'body and health',
    7: 'shopping and travel',
    8: 'online shopping',
    9: 'marketing',
    10: 'beach',
    11: 'education',
    12: 'mobile phones and devices',
    13: 'food',
    14: 'cars and driving',
    15: 'culture and colors',
    16: 'money and finances',
    17: 'energy and environment',
    18: 'house and home',
    19: 'business and technology',
}
category_to_id = {v: k for k, v in categories.items()}

In [76]:
for url in os.listdir('test'):
    
    display(Image(url=f'{base}/{url}', width=300))
    confidence = {}
    for c, prob in lda[common_dictionary.doc2bow(newEnglish[url])]:
        confidence[categories[c]] = prob
    confidence = sorted(confidence.items(), key=lambda x: (x[1],x[0]), reverse=True)
    print(confidence)

[('body and health', 0.613067), ('healthcare', 0.28624195), ('house and home', 0.058531914), ('food', 0.037858088)]


[('healthcare', 0.7217625), ('body and health', 0.18014622), ('economy and government', 0.060767878), ('cars and driving', 0.017339807), ('food', 0.011460843)]


[('energy and environment', 0.62423503), ('work and positivity', 0.14893317), ('culture and colors', 0.1276992), ('international world', 0.084317796)]


[('beach', 0.5734384), ('work and positivity', 0.19971064), ('body and health', 0.1196323), ('shopping and travel', 0.07180407), ('money and finances', 0.019114424), ('celebrations, family, and holidays', 0.011472636)]


[('international world', 0.6629067), ('mobile phones and devices', 0.20898457), ('money and finances', 0.109219804)]


...

[('food', 0.22952266), ('healthcare', 0.2089054), ('energy and environment', 0.2079299), ('international world', 0.2032337), ('house and home', 0.07190984), ('economy and government', 0.06903907)]


[('business and technology', 0.4372533), ('energy and environment', 0.42518243), ('celebrations, family, and holidays', 0.090342075)]


[('energy and environment', 0.68738234), ('beach', 0.11041014), ('international world', 0.09374459), ('economy and government', 0.05476067), ('mobile phones and devices', 0.034471516)]


[('cars and driving', 0.59259856), ('mobile phones and devices', 0.13467275), ('economy and government', 0.13119175), ('healthcare', 0.10997099), ('international world', 0.020536536)]


[('international world', 0.387795), ('economy and government', 0.24291064), ('beach', 0.16655123), ('healthcare', 0.06913412), ('money and finances', 0.04851816), ('house and home', 0.041939586), ('work and positivity', 0.024355493), ('Internet and social media', 0.011977524)]


In [90]:
image_categories = []
for url in os.listdir('test/final'):
    
    #display(Image(url=f'{base}/{url}', width=300))
    confidence = {}
    for c, prob in lda[common_dictionary.doc2bow(newEnglish[url])]:
        confidence[categories[c]] = prob
    confidence = sorted(confidence.items(), key=lambda x: (x[1],x[0]), reverse=True)
    #print(url, confidence)
    image_categories.append({
        "categories": [
            [category_to_id[cat[0]],
             cat[0]]
            for cat in confidence[:5] if category_to_id[cat[0]] not in [2, 6, 10]
        ],
        "url": url
    })
image_categories

[{'categories': [[0, 'healthcare'], [18, 'house and home'], [13, 'food']],
  'url': 'the-facts-about-heart-disease_50290aaf3b5fd.jpg'},
 {'categories': [[0, 'healthcare'],
   [3, 'celebrations and family'],
   [1, 'economy and government'],
   [13, 'food']],
  'url': 'super-dad_502910d3ad406.png'},
 {'categories': [[19, 'business and technology'],
   [7, 'shopping and travel'],
   [8, 'online shopping'],
   [9, 'marketing']],
  'url': 'hunters-vs-farmers-sales-personas_51e5924c2f823.png'},
 {'categories': [[13, 'food'],
   [3, 'celebrations and family'],
   [0, 'healthcare']],
  'url': 'calories-in-wine-vs-beer_50982977dc9eb.jpg'},
 {'categories': [[19, 'business and technology'],
   [12, 'mobile phones and devices'],
   [5, 'international world'],
   [8, 'online shopping'],
   [7, 'shopping and travel']],
  'url': 'telecom-present--the-future_530b60f629fab.jpg'},
 {'categories': [[1, 'economy and government'],
   [17, 'energy and environment'],
   [7, 'shopping and travel'],
   [14, '

In [97]:
cleaned = []
for entry in image_categories:
    clean_categories = []
    for category in entry['categories']:
        if category[0] == 0:
            clean_categories.append([0, 'health'])
        else:
            clean_categories.append(category)
    while len(clean_categories) < 5:
        cur_categories = [x[0] for x in clean_categories]
        i = 2
        while i in [2, 6, 10] or i in cur_categories:
            i = randrange(1, 19)
        clean_categories.append([i, categories[i]])    
        
    cleaned.append({
        'categories': clean_categories,
        'url': entry['url']
    }
cleaned

[{'categories': [[0, 'health'],
   [18, 'house and home'],
   [13, 'food'],
   [7, 'shopping and travel'],
   [9, 'marketing']],
  'url': 'the-facts-about-heart-disease_50290aaf3b5fd.jpg'},
 {'categories': [[0, 'health'],
   [3, 'celebrations and family'],
   [1, 'economy and government'],
   [13, 'food'],
   [17, 'energy and environment']],
  'url': 'super-dad_502910d3ad406.png'},
 {'categories': [[19, 'business and technology'],
   [7, 'shopping and travel'],
   [8, 'online shopping'],
   [9, 'marketing'],
   [5, 'international world']],
  'url': 'hunters-vs-farmers-sales-personas_51e5924c2f823.png'},
 {'categories': [[13, 'food'],
   [3, 'celebrations and family'],
   [0, 'health'],
   [14, 'cars and driving'],
   [7, 'shopping and travel']],
  'url': 'calories-in-wine-vs-beer_50982977dc9eb.jpg'},
 {'categories': [[19, 'business and technology'],
   [12, 'mobile phones and devices'],
   [5, 'international world'],
   [8, 'online shopping'],
   [7, 'shopping and travel']],
  'url': '